### Check input files



In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('../datasets'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


### Import train data and create DataFrame object

In [ ]:
rawsData = pd.read_csv('../datasets/firstDataset.csv')
print("The files have been imported")


In [ ]:
len(rawsData)


In [ ]:
rawsData.keys()

In [ ]:
rawNormal = rawsData[rawsData['Label'] == 'Benign']
rawFTP = rawsData[rawsData['Label'] == 'FTP-BruteForce']
rawSSH = rawsData[rawsData['Label'] == 'SSH-Bruteforce']

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly as py
import plotly.express as px
init_notebook_mode(connected=True)


def showAttackDistribution(data):
    normal = 'Benign'
    ftpAttack = 'FTP-BruteForce'
    sshAttack = 'SSH-Bruteforce'

    counted = data.value_counts()

    fig = go.Figure(data=[
        go.Bar(name=normal,
               y=[counted[normal]],
               x=[normal],
               text=str(counted[normal]),
               orientation='v',
               textposition='outside',),
        go.Bar(name=ftpAttack,
               y=[counted[ftpAttack]],
               x=[ftpAttack],
               text=str(counted[ftpAttack]),
               orientation='v',
               textposition='outside',),
        go.Bar(name=sshAttack,
               y=[counted[sshAttack]],
               x=[sshAttack],
               text=str(counted[sshAttack]),
               orientation='v',
               textposition='outside',)
    ])
    # Change the bar mode
    fig.update_layout(
        width=800,
        height=600,
        title=f'Labels Distribution',
        yaxis_title='Number of attacks',
        xaxis_title='Attack Name',)
    iplot(fig)


In [ ]:
showAttackDistribution(rawsData['Label'])

In [ ]:
def getSliceFromRawData():
    return rawsData[['Protocol',
                     'Fwd Pkt Len Max',
                     'Fwd Pkt Len Max',
                     'Bwd Pkt Len Mean',
                     'Fwd Pkt Len Std',
                     'Bwd Pkt Len Max',
                     'Bwd Pkt Len Min',
                     'Bwd Pkt Len Std',
                     'Bwd IAT Tot',
                     'Bwd IAT Mean',
                     'Bwd IAT Max',
                     'Pkt Len Min',
                     'Pkt Len Max',
                     'PSH Flag Cnt',
                     'Fwd Header Len',
                     'SYN Flag Cnt',
                     'RST Flag Cnt',
                     'ACK Flag Cnt',
                     'URG Flag Cnt',
                     'ECE Flag Cnt',
                     'Fwd Seg Size Avg',
                     'Init Fwd Win Byts',
                     'Fwd Act Data Pkts',
                     'Fwd Seg Size Min',
                     'Label']].copy()


In [ ]:
selectedData = getSliceFromRawData()
allDatasetToTest = getSliceFromRawData()


In [ ]:
withoutDuplicates = selectedData.drop_duplicates()


In [ ]:
len(withoutDuplicates)


### DataFrame information
#### The `info()` method provides us a basic information about featurs, like: feature's amout of data (how many values are NaN values) and the types of data (e.q int or float). It is very important information for future data processing beacuse, e.q: fetures which contains variables of type 'int' will be labeled differently than features which contains variables of type 'float'.

In [ ]:
withoutDuplicates.info()


In [ ]:
showAttackDistribution(withoutDuplicates['Label'])


In [ ]:
def shortData(data, featureName):
    # shortedNormal = data[data[featureName] == 'Benign'].sample(30000,ignore_index=True, replace=True)
    # shortedFTP = data[data[featureName] == 'FTP-BruteForce'].sample(1,ignore_index=True, replace=True)
    # shortedSSH = data[data[featureName] == 'SSH-Bruteforce'].sample(40000,ignore_index=True, replace=True)
    shortedNormal = data[data[featureName] == 'Benign'].sample(30000,ignore_index=True)
    shortedFTP = data[data[featureName] == 'FTP-BruteForce']
    shortedSSH = data[data[featureName] == 'SSH-Bruteforce'].sample(30000,ignore_index=True)
    print(len(shortedNormal))
    print(len(shortedFTP))
    print(len(shortedSSH))
    return pd.concat([shortedNormal, shortedFTP, shortedSSH], axis=0)


In [ ]:
toTrainModel = shortData(withoutDuplicates, 'Label')
showAttackDistribution(toTrainModel['Label'])


In [ ]:

def makeOnlyAttackOrNot(dataToTransform):
    columnToTransform = ['Label']
    for colKey in columnToTransform:
        dataToTransform[colKey] = dataToTransform[colKey].map(
            lambda i: 0 if i == 'Benign' else 1)
        # dataToTransform[colKey] = LabelEncoder().fit_transform(dataToTransform[colKey])

    return dataToTransform


In [ ]:
toTrainModel = makeOnlyAttackOrNot(toTrainModel)


In [ ]:
allDatasetToTest = makeOnlyAttackOrNot(allDatasetToTest)


### Heatmap
#### Heatmap provide us information on how much the individual features are interpedent. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

f, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(abs(toTrainModel.corr()), annot=True,
            linewidths=0.5, fmt='.2f', ax=ax)


Save label

In [ ]:
highest_non_inf = allDatasetToTest.max().loc[lambda v: v<np.Inf].max()
allDatasetToTest.replace(np.Inf, highest_non_inf)

In [ ]:
toTrainModel = toTrainModel.replace(np.nan, 0)
toTrainModel = toTrainModel.replace(np.inf, highest_non_inf)

allDatasetToTest = allDatasetToTest.replace(np.nan, 0)
allDatasetToTest = allDatasetToTest.replace(np.inf, highest_non_inf)


In [ ]:
labels = toTrainModel['Label']
toTrainModel = toTrainModel.drop(['Label'], axis=1)


In [ ]:
labelsToTest = allDatasetToTest['Label']
allDatasetToTest = allDatasetToTest.drop(['Label'], axis=1)


In [ ]:
# toTrainModel = toTrainModel.drop(['Fwd Seg Size Min'], axis=1)


In [ ]:
print('Is any Nan:', np.any(np.isnan(toTrainModel)))
print('Is all finite:', np.all(np.isfinite(toTrainModel)))


In [ ]:
from sklearn.preprocessing import StandardScaler


def scale(dataToTransform, columntoTransform):

    ss_dict = {col: StandardScaler() for col in columntoTransform}

    for colKey in columntoTransform:
        dataToTransform[colKey] = ss_dict[colKey].fit_transform(
            np.array(dataToTransform[colKey]).reshape(-1, 1))

    return dataToTransform


Scale numeric values

In [ ]:
# numericColumntoTransform = toTrainModel.keys()
# toTrainModel = scale(toTrainModel, numericColumntoTransform)


In [ ]:
# numericColumntoTransform = allDatasetToTest.keys()
# allDatasetToTest = scale(allDatasetToTest, numericColumntoTransform)


#### After preprocess

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    toTrainModel, labels, test_size=0.33, random_state=32)


In [ ]:
X_trainTest, X_testTest, y_trainTest, y_testTest = train_test_split(
    allDatasetToTest, labelsToTest, test_size=0.33, random_state=32)


In [ ]:
isRFC = True
isMLPC = False
isSequential = False


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense


def model():
    if isRFC:
        model = RandomForestClassifier(
            n_estimators=100, max_depth=5, min_samples_leaf=4)
    elif isMLPC:
        model = MLPClassifier(random_state=32)
    else:
        model = Sequential([
            Dense(256, activation='softplus', kernel_initializer='glorot_uniform',
                  input_dim=X_train.shape[1]),
            Dense(128, activation='softplus',
                  kernel_initializer='glorot_uniform'),
            Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')
        ])

        model.compile(loss='binary_crossentropy',
                      optimizer='Adam')

    return model


In [ ]:
model = model()


In [ ]:
model.fit(X_train, y_train)


In [ ]:

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


def confMatrix(yTrain, xTrainPreds, yTest, xTestPreds):
    # Your code here
    cmTrain = confusion_matrix(yTrain, xTrainPreds)
    X_train_disp = ConfusionMatrixDisplay(confusion_matrix=cmTrain)

    cmTest = confusion_matrix(yTest, xTestPreds)
    X_test_disp = ConfusionMatrixDisplay(confusion_matrix=cmTest)
    # End

    fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=150)

    X_train_disp.plot(ax=ax[0])
    ax[0].set_title("Train")

    X_test_disp.plot(ax=ax[1])
    ax[1].set_title("Test")
    plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

X_train_preds = model.predict(X_train)
X_test_preds = model.predict(X_test)

if isSequential:
    print('Train auc:', roc_auc_score(y_train, X_train_preds))
    print('Test auc:', roc_auc_score(y_test, X_test_preds))
else:
    print('Train accuracy: ', accuracy_score(y_train, X_train_preds))
    print('Test accuracy: ', accuracy_score(y_test, X_test_preds))


In [ ]:
confMatrix(y_train, X_train_preds, y_test, X_test_preds)


In [ ]:
X_trainTest_preds = model.predict(X_trainTest)
X_testTest_preds = model.predict(X_testTest)

if isSequential:
    print('Train auc:', roc_auc_score(y_trainTest, X_trainTest_preds))
    print('Test auc:', roc_auc_score(y_testTest, X_testTest_preds))
else:
    print('Train accuracy: ', accuracy_score(y_trainTest, X_trainTest_preds))
    print('Test accuracy: ', accuracy_score(y_testTest, X_testTest_preds))


In [ ]:
confMatrix(y_trainTest, X_trainTest_preds, y_testTest, X_testTest_preds)


In [ ]:
# toTrainModel.to_csv('test.csv', index=False)
# print("Ready")
